In [11]:
import os
import pandas as pd
import glob

from pathlib import Path

In [12]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")
table_files = glob.glob(data_dir + "/poker_table_info*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_games_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances

def get_table_df():
    table_dfs=[]
    
    for file in table_files:
        new_df = pd.read_csv(file)
        table_dfs.append(new_df)
        
    tables = pd.concat(table_dfs)
    return tables


In [13]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games_base = get_games_df()
tables = get_table_df()

games = pd.merge(games_base,tables)
hands = pd.merge(hands_base,games)

In [14]:
tables

,table_id,scenario_name,player_types
0,1,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
0,10,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
0,11,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,12,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,13,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,14,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,15,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,16,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,17,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer
0,18,smart vs 2 all call player,AlwaysCallPlayer|AlwaysCallPlayer|SmartPlayer


In [5]:
games

,table_id,game_id,player_name,player_type,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_1,AlwaysCallPlayer,lost,lost_game,small,100000,100000.0,99990.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
1,1,2,players_1,AlwaysCallPlayer,won,last_man_standing,big,100000,99990.0,100000.0,10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
2,1,3,players_1,AlwaysCallPlayer,won,won_game,small,100000,100000.0,100000.0,0.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
3,1,4,players_1,AlwaysCallPlayer,won,won_game,big,100000,100000.0,100000.0,0.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
4,1,5,players_1,AlwaysCallPlayer,won,won_game,small,100000,100000.0,100010.0,10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,9,86,players_2,SmartPlayer,lost,fold,small,100000,99970.0,99960.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
796,9,87,players_2,SmartPlayer,won,won_game,big,100000,99960.0,99960.0,0.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
797,9,88,players_2,SmartPlayer,lost,lost_game,small,100000,99960.0,99650.0,-310.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
798,9,89,players_2,SmartPlayer,lost,fold,big,100000,99650.0,99640.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer


In [6]:
hands

,table_id,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5,game_result,game_reason,blind_type,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_1,AlwaysCallPlayer,1,1,5,5,10.0,15,True,4-diamonds,Q-diamonds,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,lost,lost_game,small,100000,100000.0,99990.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
1,1,1,players_1,AlwaysCallPlayer,2,1,0,10,10.0,20,True,4-diamonds,Q-diamonds,5-clubs,6-hearts,8-spades,Z-N/A,Z-N/A,lost,lost_game,small,100000,100000.0,99990.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
2,1,1,players_1,AlwaysCallPlayer,3,1,0,10,10.0,20,True,4-diamonds,Q-diamonds,5-clubs,6-hearts,8-spades,K-spades,Z-N/A,lost,lost_game,small,100000,100000.0,99990.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
3,1,1,players_1,AlwaysCallPlayer,4,1,0,10,10.0,20,True,4-diamonds,Q-diamonds,5-clubs,6-hearts,8-spades,K-hearts,K-spades,lost,lost_game,small,100000,100000.0,99990.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
4,1,2,players_1,AlwaysCallPlayer,1,1,0,10,10.0,20,True,J-spades,Q-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,last_man_standing,big,100000,99990.0,100000.0,10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,9,89,players_2,SmartPlayer,2,1,0,10,NaN,20,True,6-diamonds,8-spades,2-clubs,9-diamonds,K-hearts,Z-N/A,Z-N/A,lost,fold,big,100000,99650.0,99640.0,-10.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
4093,9,90,players_2,SmartPlayer,1,1,5,5,10.0,15,True,2-spades,3-spades,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,won_game,small,100000,99640.0,99640.0,0.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
4094,9,90,players_2,SmartPlayer,2,1,0,10,10.0,20,True,2-spades,3-spades,3-clubs,A-hearts,Q-hearts,Z-N/A,Z-N/A,won,won_game,small,100000,99640.0,99640.0,0.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer
4095,9,90,players_2,SmartPlayer,3,1,0,10,10.0,20,True,2-spades,3-spades,10-clubs,3-clubs,A-hearts,Q-hearts,Z-N/A,won,won_game,small,100000,99640.0,99640.0,0.0,all_call_against_smart_player,AlwaysCallPlayer|SmartPlayer


# Basic Game Statistics

Number of Games played in the simulation

In [15]:
games = games[games['scenario_name'] == 'all_raise_against_smart_player']

In [16]:
number_of_games = len(set(games.game_id))
number_of_games

0

Players Mean number of chips gained and variance

In [17]:
chips_gained_per_game = games.groupby('player_name').mean('game_net_change')[['game_net_change']].reset_index()
variance_calculation = games[['player_name','game_net_change']]
variance_calculation['game_net_change'] = variance_calculation['game_net_change'] ** 2
expectation_squared = variance_calculation.groupby('player_name').mean('game_net_change').reset_index()
merged_expectations = pd.merge(expectation_squared,chips_gained_per_game,on='player_name')
merged_expectations['variance'] = merged_expectations['game_net_change_x'] - merged_expectations['game_net_change_y']
summary_stats = merged_expectations[['player_name','game_net_change_y','variance']]
summary_stats.columns = ['player_name','mean','variance']
summary_stats['games'] = number_of_games
summary_stats

,player_name,mean,variance,games


Top 10 Best Card Hands Dealt by Mean Number of Chips Gained 

In [10]:
two_card_hands = hands[['game_id','player_name','player_type','hand1','hand2','game_net_change']]
two_card_hands = two_card_hands.drop_duplicates()
best_two_card_hands = two_card_hands.groupby(['player_name','hand1','hand2']).mean().reset_index()[['player_name','hand1','hand2','game_net_change']]
best_two_card_hands = best_two_card_hands.sort_values(by=['player_name','game_net_change'], ascending=[True,False])
best_two_card_hands.groupby('player_name').head(10)

,player_name,hand1,hand2,game_net_change
72,players_1,3-spades,4-hearts,42.0
90,players_1,4-hearts,4-spades,41.0
21,players_1,10-hearts,Q-clubs,22.0
145,players_1,7-spades,Q-clubs,22.0
4,players_1,10-clubs,6-diamonds,10.0
6,players_1,10-clubs,A-spades,10.0
8,players_1,10-clubs,K-diamonds,10.0
9,players_1,10-diamonds,6-diamonds,10.0
10,players_1,10-diamonds,8-diamonds,10.0
18,players_1,10-hearts,J-clubs,10.0
